In [ ]:
## Wikipedia tool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [ ]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
## predefine tool (inbuilt tool)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)
print(wiki.name)

wikipedia


In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)

# Load Ollama embeddings 
embeddings = OllamaEmbeddings(model="mistral")  # Change to "llama2" if needed

vectordb=FAISS.from_documents(documents,embeddings)
retriever=vectordb.as_retriever()
retriever

USER_AGENT environment variable not set, consider setting it to identify your requests.


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000029C231A3AD0>, search_kwargs={})

In [ ]:
from langchain.tools.retriever import create_retriever_tool
## created custom tool
retrieval_tool=create_retriever_tool(retriever,"Langsmith_search","Search for information about langsmith")

In [6]:
retrieval_tool.name

'Langsmith_search'

In [7]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [11]:
tools=[wiki,arxiv,retrieval_tool]


In [12]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Updated-Langchain\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='Langsmith_search', description='Search for information about langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000029C144422A0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000029C231A3AD0>, search_kwa

In [14]:
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

from langchain_ollama import ChatOllama  # Import open-source LLM

# Use an open-source model like "mistral" or "llama2"
llm = ChatOllama(model="mistral", temperature=0)

# Test the model
response = llm.invoke("What is the capital of France?")
print(response)


content=' The capital of France is Paris.' additional_kwargs={} response_metadata={'model': 'mistral', 'created_at': '2025-03-05T10:13:17.6743007Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6710069500, 'load_duration': 2763679200, 'prompt_eval_count': 12, 'prompt_eval_duration': 2263091000, 'eval_count': 8, 'eval_duration': 1675984000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-1e7448df-fbc5-4045-a5a5-80450097db6e-0' usage_metadata={'input_tokens': 12, 'output_tokens': 8, 'total_tokens': 20}


In [ ]:
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate

# Define a prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    ("human", "{input}")
])

# Use an open-source model
llm = ChatOllama(model="mistral")  # Replace with "llama2", "gemma", etc.

# Format the prompt correctly
formatted_prompt = prompt.format(input="Explain LangChain")

# Run the model with the correctly formatted prompt
response = llm.invoke(formatted_prompt)
print(response)


content=" LangChain is a machine learning-powered translation service that allows developers to integrate real-time, high-quality translation of multiple languages into their applications or services. It's designed for seamless communication between different language speakers by accurately translating text and maintaining the context and tone of the original message. LangChain uses advanced Neural Machine Translation (NMT) technology to deliver fast, reliable, and contextually accurate translations. It supports a wide range of languages and can be customized based on specific industry needs for better performance." additional_kwargs={} response_metadata={'model': 'mistral', 'created_at': '2025-03-05T11:12:39.2658772Z', 'done': True, 'done_reason': 'stop', 'total_duration': 32890027500, 'load_duration': 2467014400, 'prompt_eval_count': 22, 'prompt_eval_duration': 4200999000, 'eval_count': 109, 'eval_duration': 26217852000, 'message': Message(role='assistant', content='', images=None, t

In [ ]:
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.tools import Tool
from langchain.agents import create_openai_tools_agent, AgentExecutor

# ✅ Define a valid prompt template with `agent_scratchpad`
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    ("human", "{input}"),
    ("ai", "{agent_scratchpad}")  # ✅ Required for agent to work
])

# ✅ Use an open-source model like Mistral
llm = ChatOllama(model="mistral")

# ✅ Define a sample tool
def sample_tool_function(query: str) -> str:
    return f"Tool processed query: {query}"

tools = [
    Tool(
        name="Sample Tool",
        func=sample_tool_function,
        description="A tool that processes queries."
    )
]

# ✅ Create agent with the correct prompt
agent = create_openai_tools_agent(llm, tools, prompt)

# ✅ Wrap the agent in an AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# ✅ Run the agent with an input
response = agent_executor.invoke({"input": "Explain LangChain and process 'AI' with the tool."})
print(response)




> Entering new AgentExecutor chain...
 LangChain:

LangChain is a decentralized machine learning platform built on blockchain technology. It allows developers to create, train, and deploy AI models without relying on centralized cloud services. Instead, the data and computational resources are distributed across a network of nodes, which participate in the training process and earn rewards for their contributions.

The key features of LangChain include:

1. Decentralization: The platform is built on blockchain technology to ensure decentralization and security.
2. Scalability: The architecture is designed to scale horizontally, allowing the platform to handle a large number of users and training tasks.
3. Privacy: Since data is distributed across the network, it ensures that user data remains private and secure.
4. Interoperability: LangChain supports various programming languages and AI frameworks, making it easy for developers to integrate their existing tools into the platform.
5.

In [37]:
response = agent_executor.invoke({"input": "What's the paper 1605.08386 about?"})
print(response)



> Entering new AgentExecutor chain...
1605.08386, also known as "Generating Diverse Coherent Text with Recurrent Neural Networks," is a research paper published on arXiv in May 2016. The primary focus of the paper is on text generation using recurrent neural networks (RNNs).

The authors, Dzmitry Bauza and KyungHyun Cho from Microsoft Research, propose two approaches to generate diverse, coherent text sequences:

1. A novel objective function that encourages diversity in generated text by penalizing sentences that are too similar to previously generated ones. This objective function is combined with a regularization term that rewards the model for generating longer and more informative sentences.

2. A new sampling technique called Top-k sampling, which selects the most likely words from the top k candidates at each time step during text generation. By randomly sampling among the top k options, the model can produce diverse text sequences while still maintaining a level of coherence.

In [ ]:
response = agent_executor.invoke({"input": "who was win last night cricket match  between india and austrilia?"})
print(response)



> Entering new AgentExecutor chain...
 I apologize for the confusion. The question asked about a cricket match between India and Australia that took place last night, but as of now, there is no scheduled Test or T20 series between these two teams this year, so it would be impossible to determine who won a match that did not occur. This question seems to have been made in error.

> Finished chain.
{'input': 'who was win last night in cricket match in between india and austrilia?', 'output': ' I apologize for the confusion. The question asked about a cricket match between India and Australia that took place last night, but as of now, there is no scheduled Test or T20 series between these two teams this year, so it would be impossible to determine who won a match that did not occur. This question seems to have been made in error.'}
